In [1]:
import torch.nn as nn

class YOLOShapeDetector(nn.Module):
    """
    任意個数の図形（円・三角・四角）を検出するYOLO風の軽量ネットワーク。
    1つの出力セルあたり B個の予測ボックスを出力。
    """

    def __init__(self, S=16, B=2, num_classes=3):
        """
        Args:
            S (int): 出力グリッドの分割数（S x Sセル）
            B (int): 1セルあたりの予測ボックス数
            num_classes (int): クラス数（今回は 3：円・三角・四角）
        """
        super().__init__()
        self.S = S
        self.B = B
        self.num_classes = num_classes
        self.output_dim = B * (1 + 4 + num_classes + 1)  # objectness + bbox + class probs + area

        # 軽量CNNバックボーン
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),  # 入力はグレースケール画像 (1, H, W)
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 256x256

            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 128x128

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 64x64

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32x32

            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 16x16 → S=16 に一致
        )

        # 出力層：S×SセルごとにB個の予測（objectness + bbox + class + area）
        self.pred_head = nn.Conv2d(256, self.output_dim, kernel_size=1)

    def forward(self, x):
        """
        入力画像から予測を出力

        Args:
            x (Tensor): [B, 1, H, W] のグレースケール画像

        Returns:
            Tensor: [B, S, S, B, 7 + num_classes] の予測
        """
        feat = self.features(x)  # [B, 256, S, S]
        out = self.pred_head(feat)  # [B, output_dim, S, S]

        B, C, S, S = out.shape
        out = out.permute(0, 2, 3, 1).contiguous()  # [B, S, S, output_dim]

        out = out.view(B, S, S, self.B, -1)  # [B, S, S, B, 6 + num_classes]

        return out

In [7]:
import torch

# モデル構築と重み読み込み
model = YOLOShapeDetector(S=16, B=2, num_classes=3)
model.load_state_dict(torch.load("epoch10.pth", map_location="cpu"))
model.eval()

# ダミー入力（例: 1枚のグレースケール画像 [1, 1, 512, 512]）
dummy_input = torch.randn(1, 1, 512, 512)

# ONNXとして保存
torch.onnx.export(model, dummy_input, "shape_detector.onnx",
                  input_names=["input"], output_names=["output"],
                  opset_version=11)
print("✅ ONNX形式で保存完了: shape_detector.onnx")


✅ ONNX形式で保存完了: shape_detector.onnx


C:\Users\micha\AppData\Local\Temp/ipykernel_5864/1700941501.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("epoch10.pth", map_location=

In [8]:
import onnx

# ONNXファイルを読み込む
model = onnx.load("shape_detector.onnx")

# 入力名の確認
for input_tensor in model.graph.input:
    print("入力名:", input_tensor.name)

# 出力名の確認
for output_tensor in model.graph.output:
    print("出力名:", output_tensor.name)

入力名: input
出力名: output


In [9]:
model = onnx.load("shape_detector.onnx")
print("📌 入力名:", [i.name for i in model.graph.input])
print("📌 出力名:", [o.name for o in model.graph.output])

📌 入力名: ['input']
📌 出力名: ['output']


In [2]:
import torch

# モデルのインスタンスを用意（構造が保存時と同じである必要あり）
model = YOLOShapeDetector(S=16, B=2, num_classes=3)

try:
    model.load_state_dict(torch.load("epoch10.pth", map_location="cpu"))
    print("✅ モデルの読み込みに成功しました")
except Exception as e:
    print("❌ 読み込み失敗:", e)

✅ モデルの読み込みに成功しました


C:\Users\micha\AppData\Local\Temp/ipykernel_3960/2863949496.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("epoch10.pth", map_location=

In [3]:
state_dict = torch.load("epoch10.pth", map_location="cpu")
for k, v in state_dict.items():
    print(f"{k}: {v.shape}")

features.0.weight: torch.Size([16, 1, 3, 3])
features.0.bias: torch.Size([16])
features.1.weight: torch.Size([16])
features.1.bias: torch.Size([16])
features.1.running_mean: torch.Size([16])
features.1.running_var: torch.Size([16])
features.1.num_batches_tracked: torch.Size([])
features.4.weight: torch.Size([32, 16, 3, 3])
features.4.bias: torch.Size([32])
features.5.weight: torch.Size([32])
features.5.bias: torch.Size([32])
features.5.running_mean: torch.Size([32])
features.5.running_var: torch.Size([32])
features.5.num_batches_tracked: torch.Size([])
features.8.weight: torch.Size([64, 32, 3, 3])
features.8.bias: torch.Size([64])
features.9.weight: torch.Size([64])
features.9.bias: torch.Size([64])
features.9.running_mean: torch.Size([64])
features.9.running_var: torch.Size([64])
features.9.num_batches_tracked: torch.Size([])
features.12.weight: torch.Size([128, 64, 3, 3])
features.12.bias: torch.Size([128])
features.13.weight: torch.Size([128])
features.13.bias: torch.Size([128])
fea

C:\Users\micha\AppData\Local\Temp/ipykernel_3960/1147199462.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("epoch10.pth", map_location="cpu")


In [4]:
model.eval()
dummy_input = torch.randn(1, 1, 512, 512)  # 入力サイズと同じテンソル

with torch.no_grad():
    output = model(dummy_input)
    print("✅ 推論成功。出力サイズ:", output["output"].shape if isinstance(output, dict) else output.shape)

✅ 推論成功。出力サイズ: torch.Size([1, 16, 16, 2, 9])


In [5]:
print(output)

tensor([[[[[ 5.1458,  1.1833,  1.0737,  ..., -2.5919, -2.8942, -0.1407],
           [-6.2964,  1.9910,  1.1692,  ..., -5.1240, -4.8229, -3.3993]],

          [[ 8.4656,  0.7383,  0.9216,  ...,  0.7699, -3.2333, -0.1409],
           [-0.1202,  1.2647,  0.7411,  ..., -4.2575, -3.4888, -2.7104]],

          [[10.0604,  0.7015,  0.9796,  ...,  0.9111, -3.3052, -0.1413],
           [ 0.4313,  1.1894,  0.7369,  ..., -4.6759, -3.7304, -2.7940]],

          ...,

          [[ 9.0369,  0.7060,  0.9295,  ...,  0.0407, -2.7553, -0.1235],
           [ 0.3575,  1.1347,  0.7501,  ..., -4.4404, -3.3162, -2.5226]],

          [[ 9.3854,  0.6630,  0.9838,  ...,  0.4747, -3.2442, -0.1290],
           [ 0.1025,  1.1470,  0.6961,  ..., -4.7163, -3.5064, -2.7031]],

          [[ 5.3006,  0.1975,  1.0837,  ...,  3.8709, -3.6939, -0.1458],
           [-5.1387,  0.9585,  1.3445,  ..., -4.7916, -2.6800, -2.9720]]],


         [[[ 9.1355,  1.1770,  0.4572,  ..., -5.0513, -3.0504, -0.1337],
           [ 0.7753, 

In [6]:
torch.onnx.export(model, dummy_input, "check.onnx",
                  input_names=["input"], output_names=["output"],
                  opset_version=11)
print("✅ ONNX変換も成功")

✅ ONNX変換も成功
